In [3]:
using Oscar

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | Version 1.4.1


In [4]:
# defines the denominator
sl2size(p, k) = p^(2*k - 2) * (p^2 - 1)

sl2size (generic function with 1 method)

In [19]:
# Assume all matrices are of the form (a b c d)
function ratio1(p, n, trace, det)
    count = p^(2*n-1) * (p-1) # number of matrices such that p does not divides b
    for a in 0:(p^n-1)
        rhs = a*(trace - a) - det
        if rhs % p^n != 0      
            count += valuation(rhs, p) * p^(n-1)*(p-1) # number of pairs (b,c) that satisfies bc = ad - det mod p^n
        else
            count += p^n # number of matrices with b = 0
            count += (n-1)*(p^n - p^(n-1)) # b > 0
        end
    end
    return count
end

ratio1 (generic function with 1 method)

In [28]:
# this function generate all matrices satisfying ratio1
function build_matrices1(p, n, trace, det)
    Rn, = residue_ring(ZZ, p^n)
    result = Vector{NTuple{4, zzModRingElem}}()

    for a in Rn
        d = trace - a
        rhs = Rn(a*d - det)

        # matrices with gcd(b, p) = 1
        for b in Rn
            if lift(b) % p != 0
                push!(result, (a, b, rhs/b, d))
            end
        end

        # matrices with vp(b) >= 1
        if rhs != 0
            rhs_lift = lift(rhs) # convert rhs to integer
            pval = valuation(rhs_lift, p)
            for exp in 1:pval
                rhs_reduced = Rn(rhs_lift / p^exp)
                for j in 1:(p^(n-exp))-1
                    if j % p != 0
                        b = Rn(p^exp * j)
                        c_start = lift(rhs_reduced / j) % p^(n-exp)
                        for c in c_start: p^(n-exp): p^n-1
                            push!(result, (a, b, Rn(c), d))
                        end
                    end
                end
            end
        else
            for c in Rn
                push!(result, (a, Rn(0), c, d))
            end

            for exp in 1:n-1
                for j in 1:(p^(n-exp) - 1)
                    if j % p != 0
                        b = Rn(p^exp * j)
                        for c in 0:p^(n-exp):p^n-1
                            push!(result, (a, b, Rn(c), d))
                        end
                    end
                end
            end
        end
    end

    return result
end

build_matrices1 (generic function with 1 method)

In [29]:
@time println(ratio1(3, 5, 5, 4))
@time println(length(build_matrices1(3, 5, 5, 4)))

78732
  0.000267 seconds (735 allocations: 11.586 KiB)
78732
  0.224469 seconds (328.83 k allocations: 19.644 MiB, 66.46% gc time)


In [35]:
function ratio2_adder(p, n, kbig, projset, rhs_new, a, ind, exp, Rn)
    c_start = lift(rhs_new / ind) % p^(kbig-exp)
    if (Rn(a), Rn(c_start)) in projset
        return false
    end

    push!(projset, (Rn(a), Rn(c_start)))
    for c in (c_start + p^(kbig-exp)):p^(kbig-exp):(p^n-1)
        push!(projset, (Rn(a), Rn(c)))
    end
    return true
end

function ratio2_adder_with_check(p, n, kbig, projset, lift_dict, rhs_new, a, ind, exp, Rn, Rkbig)
    c_start = mod(lift(rhs_new / ind), p^(kbig-exp))
    key = (Rn(a), Rn(c_start))
    if key in projset
        lifted = lift_dict[key]
        if iszero(lifted[2]) || valuation(lift(lifted[2]), p) == exp
            return false
        end
    end

    push!(projset, key)
    lift_dict[key] = (Rkbig(a), Rkbig(p^exp * ind), Rkbig(c_start))
    for c in (c_start + p^(kbig-exp)) : p^(kbig-exp) : (p^n-1)
        lift_dict[(Rn(a), Rn(c))] = (Rkbig(a), Rkbig(p^exp * ind), Rkbig(c))
        push!(projset, (Rn(a), Rn(c)))
    end
    return true
end

ratio2_adder_with_check (generic function with 2 methods)

In [38]:
function ratio2(p, kbig, n, trace, det)
    if n >= kbig
        println("Must project from big ring to small ring")
        return nothing
    end

    count = p^(2*n - 1) * (p-1) # number of matrices such that p does not divides b (is not affected by the projection)
    Rn, = residue_ring(ZZ, p^n)
    Rkbig, = residue_ring(ZZ, p^kbig)

    for exp in 1:(n-1)
        # no need to keep track of b since it is fixed to b=p^exp mod p^n, of d as well, since it just follows from a
        projset = Set{Tuple{zzModRingElem, zzModRingElem}}() 
        for a in 0:(p^kbig - 1)
            d = Rkbig(trace - a)
            rhs = Rkbig(a * d - det)

            if !iszero(rhs)
                pval = valuation(lift(rhs), p)
                if pval >= exp
                    rhs_reduced = Rkbig(lift(rhs) / p^exp)
                    for ind in 1: p^(n-exp): p^(kbig - exp) 
                        if !ratio2_adder(p, n, kbig, projset, rhs_reduced, a, ind, exp, Rn)
                            break
                        end
                    end
                end
            else
                ratio2_adder(p, n, kbig, projset, Rkbig(0), a, 1, exp, Rn)
            end
        end
        count += length(projset) * (p^(n-exp) - p^(n-exp-1)) # all b with vp(b) = exp also have the same count
    end

    # below is for the case when b = 0 mod p^n
    projset = Set{Tuple{zzModRingElem, zzModRingElem}}()
    
    # lift_dict is a dictionary to keep track of all matrices with their corresponding lift 
    # (so that the some simplifications explained in the pdf file can be applied)
    lift_dict = Dict{Tuple{zzModRingElem, zzModRingElem}, Tuple{zzModRingElem, zzModRingElem, zzModRingElem}}() 
    for a in 0:(p^kbig - 1)
        d = Rkbig(trace - a)
        rhs = Rkbig(a*d - det)

        if !iszero(rhs)
            pval = valuation(lift(rhs), p)
            for exp in n:pval
                rhs_reduced = Rkbig(lift(rhs) / p^exp)
                for j in 1:p^(kbig - exp)
                    if j % p != 0 && !ratio2_adder_with_check(p, n, kbig, projset, lift_dict, rhs_reduced, a, j, exp, Rn, Rkbig)
                        break
                    end
                end
            end
        else
            ratio2_adder_with_check(p, n, kbig, projset, lift_dict, Rkbig(0), a, 1, kbig, Rn, Rkbig)
        end
    end
    return count + length(projset)
end

ratio2 (generic function with 1 method)

In [39]:
@time println(ratio2(3, 13, 6, 5, 4))

551124
  8.402528 seconds (41.55 M allocations: 638.714 MiB, 43.56% gc time)


In [11]:
function compute_ratio1_ratio2(p, n, trace, det)
    # Ratio 1
    num1 = ratio1(p, n, trace, det)
    ratio1_val = num1 // sl2size(p, n)
    println("ratio 1: ", ratio1_val)
    
    # Ratio 2
    kbig_bound = 12
    num2 = Dict{Int, Int}()
    for kbig in (n+1):kbig_bound
        num2[kbig - n] = ratio2(p, kbig, n, trace, det)
    end
    ratio2_res = [num2[k] // sl2size(p, n) for k in 1:(kbig_bound - n)]
    println("ratio 2: ", ratio2_res)
end

compute_ratio1_ratio2 (generic function with 1 method)

In [12]:
@time compute_ratio1_ratio2(3, 5, 5, 4)

ratio 1: 3//2
ratio 2: Rational{Int64}[7//6, 7//6, 7//6, 7//6, 7//6, 7//6, 7//6]
  5.273321 seconds (18.74 M allocations: 304.186 MiB, 37.92% gc time, 12.92% compilation time)
